# Mise en forme des données et intégration dans une base de donnée

In [111]:
import pandas as pd
from datetime import datetime
import json

## 1. Charger et examiner les fichiers

### 1.1 Chargement des csv.

#### DataFrame matches

In [112]:
#df_2018 = pd.read_json('./data/data-2018.json')
df_matches = pd.read_csv('./data/matches.csv')

Harmonisation des colonnes de df_matches

In [113]:
df_matches[['home_result', 'away_result']] = df_matches['score'].str.extract(r'(\d+)-(\d+)').astype(float)

df_matches = df_matches.rename(columns={
    'team1': 'home_team',
    'team2': 'away_team',
    'venue': 'city',
    'edition': 'edition',
    'round': 'round',
    'year': 'edition_year'
})

df_matches['id_match'] = df_matches.index + 1

df_matches = df_matches[['id_match', 'home_team', 'away_team', 'home_result', 'away_result', 'edition', 'round', 'city', 'edition_year']]

df_matches['city'] = df_matches['city'].str.rstrip('.')

In [114]:
df_matches

,id_match,home_team,away_team,home_result,away_result,edition,round,city,edition_year
0,1,France,Mexico (México),4.0,1.0,1930-URUGUAY,GROUP_STAGE,Montevideo,1930
1,2,USA,Belgium (België),3.0,0.0,1930-URUGUAY,GROUP_STAGE,Montevideo,1930
2,3,Yugoslavia (Југославија),Brazil (Brasil),2.0,1.0,1930-URUGUAY,GROUP_STAGE,Montevideo,1930
3,4,Romania (România),Peru (Perú),3.0,1.0,1930-URUGUAY,GROUP_STAGE,Montevideo,1930
4,5,Argentina,France,1.0,0.0,1930-URUGUAY,GROUP_STAGE,Montevideo,1930
...,...,...,...,...,...,...,...,...,...
7294,7295,3,4,NaN,NaN,2014-BRAZIL,1/4_FINAL,Salvador,2014
7295,7296,A,B,NaN,NaN,2014-BRAZIL,1/2_FINAL,Belo Horizonte,2014
7296,7297,C,D,NaN,NaN,2014-BRAZIL,1/2_FINAL,São_Paulo,2014
7297,7298,LOSER X,LOSER Y,NaN,NaN,2014-BRAZIL,PLACES_3&4,Brasília,2014


#### DataFrame 2014

In [115]:
df_2014 = pd.read_csv('./data/worldcupmatches2014.csv', sep=';')

In [116]:
df_2014 = df_2014.rename(columns={
    'Home Team Name': 'home_team',
    'Away Team Name': 'away_team',
    'Home Team Goals': 'home_result',
    'Away Team Goals': 'away_result',
    'City': 'city',
    'Stage': 'round',
    'Year': 'edition_year'
})

df_2014['edition'] = '2014'
df_2014 = df_2014[['home_team', 'away_team', 'home_result', 'away_result', 'edition', 'round', 'city', 'edition_year']]

In [117]:
df_2014

,home_team,away_team,home_result,away_result,edition,round,city,edition_year
0,Brazil,Croatia,3,1,2014,Group A,Sao Paulo,2014
1,Mexico,Cameroon,1,0,2014,Group A,Natal,2014
2,Spain,Netherlands,1,5,2014,Group B,Salvador,2014
3,Chile,Australia,3,1,2014,Group B,Cuiaba,2014
4,Colombia,Greece,3,0,2014,Group C,Belo Horizonte,2014
...,...,...,...,...,...,...,...,...
75,Netherlands,Costa Rica,0,0,2014,Quarter-finals,Salvador,2014
76,Brazil,Germany,1,7,2014,Semi-finals,Belo Horizonte,2014
77,Netherlands,Argentina,0,0,2014,Semi-finals,Sao Paulo,2014
78,Brazil,Netherlands,0,3,2014,Play-off for third place,Brasilia,2014


#### DataFrame 2022

In [118]:
df_2022 = pd.read_csv('./data/Fifa_world_cup_matches_2022.csv')

In [119]:
df_2022 = df_2022.rename(columns={
    'team1': 'home_team',
    'team2': 'away_team',
    'number of goals team1': 'home_result',
    'number of goals team2': 'away_result',
    'category': 'round',
    'date': 'edition_year'
})

df_2022['edition'] = '2022'
df_2022['edition_year'] = 2022
df_2022['round'] = df_2022['round'].fillna('Unknown')
df_2022 = df_2022[['home_team', 'away_team', 'home_result', 'away_result', 'edition', 'round', 'edition_year']]

In [120]:
df_2022

,home_team,away_team,home_result,away_result,edition,round,edition_year
0,QATAR,ECUADOR,0,2,2022,Group A,2022
1,ENGLAND,IRAN,6,2,2022,Group B,2022
2,SENEGAL,NETHERLANDS,0,2,2022,Group A,2022
3,UNITED STATES,WALES,1,1,2022,Group B,2022
4,ARGENTINA,SAUDI ARABIA,1,2,2022,Group C,2022
...,...,...,...,...,...,...,...
59,ENGLAND,FRANCE,1,2,2022,Quarter-final,2022
60,ARGENTINA,CROATIA,3,0,2022,Semi-final,2022
61,FRANCE,MOROCCO,2,0,2022,Semi-final,2022
62,CROATIA,MOROCCO,2,1,2022,Play-off for third place,2022


### 1.2 Cas particulier du json

In [121]:
with open('./data/data-2018.json') as f:
    data_2018 = json.load(f)

In [122]:
data_2018

{'stadiums': [{'id': 1,
   'name': 'Luzhniki Stadium',
   'city': 'Moscow',
   'lat': 55.715765,
   'lng': 37.5515217,
   'image': 'https://upload.wikimedia.org/wikipedia/commons/e/e6/Luzhniki_Stadium%2C_Moscow.jpg'},
  {'id': 2,
   'name': 'Otkrytiye Arena',
   'city': 'Moscow',
   'lat': 55.817765,
   'lng': 37.440363,
   'image': 'https://upload.wikimedia.org/wikipedia/commons/5/50/Stadium_Spartak_in_Moscow.jpg'},
  {'id': 3,
   'name': 'Krestovsky Stadium',
   'city': 'Saint Petersburg',
   'lat': 59.97274,
   'lng': 30.221408,
   'image': 'https://upload.wikimedia.org/wikipedia/commons/a/ad/Spb_06-2017_img40_Krestovsky_Stadium.jpg'},
  {'id': 4,
   'name': 'Kaliningrad Stadium',
   'city': 'Kaliningrad',
   'lat': 54.698157,
   'lng': 20.533859,
   'image': 'https://upload.wikimedia.org/wikipedia/commons/a/a4/Kaliningrad_stadium_-_2018-04-07.jpg'},
  {'id': 5,
   'name': 'Kazan Arena',
   'city': 'Kazan',
   'lat': 55.820983,
   'lng': 49.160966,
   'image': 'https://upload.wikime

Extraire les données des matchs de 2018

In [123]:
matches_2018 = []
for group, data in data_2018['groups'].items():
    for match in data['matches']:
        matches_2018.append({
            'home_team': data_2018['teams'][match['home_team'] - 1]['name'],
            'away_team': data_2018['teams'][match['away_team'] - 1]['name'],
            'home_result': match['home_result'],
            'away_result': match['away_result'],
            'round': 'Group Stage',
            'edition': '2018',
            'edition_year': 2018
        })

Extraire les matchs de phase finale (knockout)

In [124]:
for round_name, data in data_2018['knockout'].items():
    for match in data['matches']:
        round_label = data['name']
        matches_2018.append({
            'home_team': data_2018['teams'][match['home_team'] - 1]['name'],
            'away_team': data_2018['teams'][match['away_team'] - 1]['name'],
            'home_result': match['home_result'],
            'away_result': match['away_result'],
            'round': round_label,
            'edition': '2018',
            'edition_year': 2018
        })

In [125]:
df_2018 = pd.DataFrame(matches_2018)

In [126]:
df_2018

,home_team,away_team,home_result,away_result,round,edition,edition_year
0,Russia,Saudi Arabia,5,0,Group Stage,2018,2018
1,Egypt,Uruguay,0,1,Group Stage,2018,2018
2,Russia,Egypt,3,1,Group Stage,2018,2018
3,Uruguay,Saudi Arabia,1,0,Group Stage,2018,2018
4,Uruguay,Russia,3,0,Group Stage,2018,2018
...,...,...,...,...,...,...,...
59,Sweden,England,0,2,Quarter-finals,2018,2018
60,France,Belgium,1,0,Semi-finals,2018,2018
61,Croatia,England,2,1,Semi-finals,2018,2018
62,Belgium,England,2,0,Third place play-off,2018,2018


## 2. Fusion des datasets

In [127]:
df_all_matches = pd.concat([df_matches, df_2014, df_2018, df_2022], ignore_index=True)

In [128]:
df_all_matches

,id_match,home_team,away_team,home_result,away_result,edition,round,city,edition_year
0,1.0,France,Mexico (México),4.0,1.0,1930-URUGUAY,GROUP_STAGE,Montevideo,1930
1,2.0,USA,Belgium (België),3.0,0.0,1930-URUGUAY,GROUP_STAGE,Montevideo,1930
2,3.0,Yugoslavia (Југославија),Brazil (Brasil),2.0,1.0,1930-URUGUAY,GROUP_STAGE,Montevideo,1930
3,4.0,Romania (România),Peru (Perú),3.0,1.0,1930-URUGUAY,GROUP_STAGE,Montevideo,1930
4,5.0,Argentina,France,1.0,0.0,1930-URUGUAY,GROUP_STAGE,Montevideo,1930
...,...,...,...,...,...,...,...,...,...
7502,NaN,ENGLAND,FRANCE,1.0,2.0,2022,Quarter-final,NaN,2022
7503,NaN,ARGENTINA,CROATIA,3.0,0.0,2022,Semi-final,NaN,2022
7504,NaN,FRANCE,MOROCCO,2.0,0.0,2022,Semi-final,NaN,2022
7505,NaN,CROATIA,MOROCCO,2.0,1.0,2022,Play-off for third place,NaN,2022


## 3. Export des données

In [129]:
df_all_matches.to_csv('./data/data_final.csv', index=False)
print("Le fichier final 'data_final.csv' a été créé avec succès.")

Le fichier final 'data_final.csv' a été créé avec succès.


## 4. Analyses

1) Identifier le pays ayant gagné le plus de matchs et le nombre de victoires

In [130]:
df_all_matches['winner'] = df_all_matches.apply(
    lambda row: row['home_team'] if row['home_result'] > row['away_result'] else (
        row['away_team'] if row['away_result'] > row['home_result'] else None
    ), axis=1
)
top_country_wins = df_all_matches['winner'].value_counts().idxmax()
top_country_wins_count = df_all_matches['winner'].value_counts().max()

print(f"Le pays ayant gagné le plus de matchs est : {top_country_wins} avec {top_country_wins_count} victoires.")

Le pays ayant gagné le plus de matchs est : Brazil (Brasil) avec 123 victoires.


2) Top 5 des vainqueurs de la Coupe du Monde (finale) :

In [131]:
# Filtrer les matchs de la finale
final_matches = df_all_matches[df_all_matches['round'] == 'Final'].copy()

# Ajouter une colonne pour le gagnant dans les finales
final_matches.loc[:, 'winner'] = final_matches.apply(
    lambda row: row['home_team'] if row['home_result'] > row['away_result'] else (
        row['away_team'] if row['away_result'] > row['home_result'] else None
    ), axis=1
)

# Calculer le Top 5 des vainqueurs de la finale avec le nombre de victoires
top_final_winners = final_matches['winner'].value_counts().head(5)

# Afficher le résultat
print("Top 5 des vainqueurs de la Coupe du Monde (finale) :")
print(top_final_winners)

Top 5 des vainqueurs de la Coupe du Monde (finale) :
winner
Germany    2
France     1
Name: count, dtype: int64


3) Plus grand écart de buts dans un match

In [ ]:
df_all_matches['goal_diff'] = abs(df_all_matches['home_result'] - df_all_matches['away_result'])

max_goal_diff = df_all_matches['goal_diff'].max()

matches_with_max_goal_diff = df_all_matches[df_all_matches['goal_diff'] == max_goal_diff]

print("Le plus grand écart de buts dans un match est :", max_goal_diff)
print("Match avec le plus grand écart de buts :")
print(matches_with_max_goal_diff[['home_team', 'away_team', 'home_result', 'away_result', 'edition', 'round', 'edition_year']])

Le plus grand écart de buts dans un match est : 31.0
Match(s) avec le plus grand écart de buts :
      home_team       away_team  home_result  away_result           edition  \
4160  Australia  American Samoa         31.0          0.0  2002-KOREA/JAPAN   

                    round  edition_year  
4160  PRELIMINARY-Oceania          2001  


4) Nombre total de matchs nuls 

In [133]:
total_draws = df_all_matches[df_all_matches['home_result'] == df_all_matches['away_result']].shape[0]

print("Le nombre total de matchs nuls est :", total_draws)

Le nombre total de matchs nuls est : 1595


5) Pays ayant accédé le plus souvent aux phases finales

In [134]:
phase_finale = df_all_matches[df_all_matches['round'].isin(['Round of 16', 'Quarter-finals', 'Semi-finals', 'Final'])]

top_country_finale = pd.concat([phase_finale['home_team'], phase_finale['away_team']]).value_counts().idxmax()
top_country_finale_count = pd.concat([phase_finale['home_team'], phase_finale['away_team']]).value_counts().max()

print(f"Le pays ayant accédé le plus souvent aux phases finales est : {top_country_finale}")
print(f"Il y a accédé {top_country_finale_count} fois.")

Le pays ayant accédé le plus souvent aux phases finales est : Argentina
Il y a accédé 9 fois.
